# Examples with Partitions

> **These are Go notebooks**: In order to use the GoNB Jupyter Kernel, please install GoNB from here: [https://github.com/janpfeifer/gonb](https://github.com/janpfeifer/gonb)

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/stochadex=/path/to/stochadex"` at the top of any cell.

## UNIX timestamps

Running the simulator with UNIX timestamps representing the time variable makes plotting correctly formatted date-time strings possible with `go-echarts`. Here's a complete example using some simple Wiener processes below.

In [ ]:
import (
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	"github.com/umbralcalc/stochadex/pkg/continuous"

	"github.com/go-echarts/go-echarts/v2/opts"
	"github.com/go-echarts/go-echarts/v2/charts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

// Create a simulator.StateTimeStorage from a simulation run
storage := analysis.NewStateTimeStorageFromPartitions(
	// Instantiate the desired simulation state partitions
	[]*simulator.PartitionConfig{{
		Name:              "test_partition",
		Iteration:         &continuous.WienerProcessIteration{},
		Params:            simulator.NewParams(map[string][]float64{
			"variances": {1.0, 2.0, 3.0, 4.0},
		}),
		InitStateValues:   []float64{0.0, 0.0, 0.0, 0.0},
		StateHistoryDepth: 1,
		Seed:              12345,
	}},
	// Decide when should we stop the simulation
	&simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 100,
	},
	// Decide how time should evolve
	&simulator.ConstantTimestepFunction{
		Stepsize: 1000.0,
	},
	// Input the initial time
	1667980544.0,
)

// Reference the plotting data for the x-axis
xRef := analysis.DataRef{PartitionName: "test_partition", IsTime: true}

// Reference the plotting data for the y-axis
yRefs = []analysis.DataRef{{
	PartitionName: "test_partition",
	ValueIndices: []int{0, 1, 2, 3},
}}

// Create a scatter plot from partitions in a simulator.StateTimeStorage
scatter := analysis.NewScatterPlotFromPartition(storage, xRef, yRefs)

// Display date-time strings when the time is a UNIX timestamp
scatter.SetGlobalOptions(charts.WithXAxisOpts(opts.XAxis{Type: "time"}))

// Display the plot in a Go notebook
gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")

## Custom iterations

Defining a custom state partition iteration just requires an implementation of the `simulator.Iteration` interface with a new struct. These iterations can then immediately be run to generate new data like the binomial example below.

In [ ]:
import (
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	"github.com/umbralcalc/stochadex/pkg/continuous"

	"golang.org/x/exp/rand"
	"gonum.org/v1/gonum/stat/distuv"
	"github.com/go-echarts/go-echarts/v2/opts"
	"github.com/go-echarts/go-echarts/v2/charts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

// Create a new partition iteration struct
type MyCustomIteration struct {
	binomialDist *distuv.Binomial
}

// Define how the parameters and settings are used configure this iteration
func (m *MyCustomIteration) Configure(
	partitionIndex int,
	settings *simulator.Settings,
) {
	m.binomialDist = &distuv.Binomial{
		N:   0,
		P:   1.0,
		Src: rand.NewSource(settings.Seeds[partitionIndex]),
	}
}

// Define how this iteration actually changes the state of the partition over time
func (m *MyCustomIteration) Iterate(
	params *simulator.Params,
	partitionIndex int,
	stateHistories []*simulator.StateHistory,
	timestepsHistory *simulator.CumulativeTimestepsHistory,
) []float64 {
	outputValues := make([]float64, 0)
	ps := params.Get("p_values")
	for i, n := range params.Get("n_values") {
		m.binomialDist.N = n
		m.binomialDist.P = ps[i]
		outputValues = append(outputValues, m.binomialDist.Rand())
	}
	return outputValues
}

%%

// Create a simulator.StateTimeStorage from a simulation run
storage := analysis.NewStateTimeStorageFromPartitions(
	// Instantiate the desired simulation state partitions
	[]*simulator.PartitionConfig{{
		Name:              "custom_partition",
		Iteration:         &MyCustomIteration{},
		Params:            simulator.NewParams(map[string][]float64{
			"n_values": {10, 14, 27},
			"p_values": {0.3, 0.8, 0.1},
		}),
		InitStateValues:   []float64{0.0, 0.0, 0.0},
		StateHistoryDepth: 1,
		Seed:              3421,
	}},
	// Decide when should we stop the simulation
	&simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 100,
	},
	// Decide how time should evolve
	&simulator.ConstantTimestepFunction{
		Stepsize: 1.0,
	},
	// Input the initial time
	0.0,
)

// Reference the plotting data for the x-axis
xRef := analysis.DataRef{PartitionName: "custom_partition", IsTime: true}

// Reference the plotting data for the y-axis
yRefs := []analysis.DataRef{{
	PartitionName: "custom_partition",
	ValueIndices: []int{0, 1, 2},
}}

// Create a scatter plot from partitions in a simulator.StateTimeStorage
scatter := analysis.NewScatterPlotFromPartition(storage, xRef, yRefs)

// Display the plot in a Go notebook
gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")